In [1]:
# import packages
import pandas as pd # for dataframes
from geopy.geocoders import Nominatim # for reverse geocoding
import requests # for getting external data tables

In [2]:
# load dataframes
df_19 = pd.read_csv("Airbnb_NYC_2019.csv")
df_20 = pd.read_csv("listings.csv")

In [3]:
# use only common columns
# identify
cols_19 = df_19.columns._data
cols_20 = df_20.columns._data
common_cols = []

for col in cols_19:
    if col in cols_20:
        common_cols.append(col)

# extract
df_19 = df_19[common_cols]
df_20 = df_20[common_cols]

In [4]:
# clean prices
# change price from string to float
df_19.loc[:,'price'] = df_19.price.replace({'\$': '', ',': ''}, regex=True).astype(float)
df_20.loc[:,'price'] = df_20.price.replace({'\$': '', ',': ''}, regex=True).astype(float)

# remove listings with price greater than $5000/night
df_19 = df_19[df_19.price < 1000]
df_20 = df_20[df_20.price < 1000]

# reset indices
df_19 = df_19.reset_index(drop=True)
df_20 = df_20.reset_index(drop=True)

In [5]:
# get only 2019 rated listings
# find old listings
review_19 = []
review_20 = []

# 2019
val_years_19 = '2019'
for i in range(len(df_19)):
    val = val_years_19 in str(df_19.last_review.loc[i])
    review_19.append(val)
    
# 2020
val_years_20 = '2020'
for i in range(len(df_20)):
    val = val_years_20 in str(df_20.last_review.loc[i])
    review_20.append(val)

# remove
df_pre = df_19[review_19]
df_post = df_20[review_20]

# reset indices
df_pre = df_pre.reset_index(drop=True)
df_post = df_post.reset_index(drop=True)

In [6]:
# remove listings that have 'hidden by airbnb'
# find hidden listings
pre_inds = []
post_inds = []
# pre
for i in range(len(df_pre)):
    if 'hidden by airbnb' in str(df_pre.name[i]).lower():
        print(df_pre.name[i])
        pre_inds.append(i)
        
# post
for i in range(len(df_post)):
    if 'hidden by airbnb' in str(df_post.name[i]).lower():
        print(df_post.name[i])
        post_inds.append(i)
        
# remove
df_pre = df_pre.drop(index=pre_inds)
df_post = df_post.drop(index=post_inds)

# reset indices
df_pre = df_pre.reset_index(drop=True)
df_post = df_post.reset_index(drop=True)

(Hidden by Airbnb) ! BrooklynCleanRoom!!
浪漫民宿,环境优美,停车方便,独立洗手间和马桶,浴室共用｡ (Website hidden by Airbnb)
阳光民宿 单房干净舒适｡“和缘国旅”了解更多旅游､民宿 (Website hidden by Airbnb)
1BR/Studio Superb for a professional (Hidden by Airbnb) Home
SPECTACULAR 2br high floor (Website hidden by Airbnb) view
Spacious Park Avenue (Website hidden by Airbnb) -2-Month lease
E community that is commercially (Website hidden by Airbnb) smoke
Nolita! Cute one bedroom (Website hidden by Airbnb) Location!


In [7]:
# mark postcodes by income bracket
# get income data
rich_url = 'https://www.zipdatamaps.com/economics/income/agi/state/wealthiest-zipcodes-in-new-york'
rich_codes = pd.read_html(requests.get(rich_url).content)[0]['List of 25 Wealthiest Zipcodes in New York'].Zipcode.tolist()
for i in [6,12-1,18-2]:
    rich_codes.pop(i)

In [8]:
# get counties and neighbourhoods for each listing

In [9]:
# function to add postcode column to df
def get_postcodes(df,geom_list,geom_inds):
    # get locator
    locator = Nominatim(user_agent="myGeocoder",timeout=None)
    # add postcodes 
    for i in geom_inds:
        if i % 500 == 0:
            print("reverse geocoding listing {}".format(i))
            # get locator
            # locator = Nominatim(user_agent="myGeocoder")
        address = locator.reverse(geom_list[i]).raw['address']
        if 'postcode' in address.keys():
            df.postcode.iloc[i] = address['postcode']
            if address['postcode'] in rich_codes:
                df.rich.iloc[i] = 1
            else:
                df.rich.iloc[i] = 0
        else:
            print("no postcode for listing {}".format(i))
                
        if 'suburb' in address.keys():
            df.borough.iloc[i] = address['suburb']
        else:
            print("no borough for listing {}".format(i))
        
    return df

In [10]:
# add columns
df_pre['postcode'] = 'NA'
df_pre['rich'] = 'NA'
df_pre['borough'] = 'NA'
df_post['postcode'] = 'NA'
df_post['rich'] = 'NA'
df_post['borough'] = 'NA'

In [11]:
# get coordinates list
geom_list_pre = df_pre.latitude.map(str) + ", " + df_pre.longitude.map(str)
geom_list_post = df_post.latitude.map(str) + ", " + df_post.longitude.map(str)

In [12]:
df_pre = get_postcodes(df_pre, geom_list_pre, range(len(df_pre)))

reverse geocoding listing 0


C:\Users\samih\anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


no postcode for listing 159
reverse geocoding listing 500
reverse geocoding listing 1000
no postcode for listing 1313
reverse geocoding listing 1500
reverse geocoding listing 2000
reverse geocoding listing 2500
reverse geocoding listing 3000
reverse geocoding listing 3500
reverse geocoding listing 4000
reverse geocoding listing 4500
reverse geocoding listing 5000
no postcode for listing 5170
reverse geocoding listing 5500
reverse geocoding listing 6000
reverse geocoding listing 6500
reverse geocoding listing 7000
reverse geocoding listing 7500
reverse geocoding listing 8000
reverse geocoding listing 8500
no postcode for listing 8744
reverse geocoding listing 9000
no postcode for listing 9252
reverse geocoding listing 9500
reverse geocoding listing 10000
reverse geocoding listing 10500
reverse geocoding listing 11000
reverse geocoding listing 11500
no postcode for listing 11774
reverse geocoding listing 12000
reverse geocoding listing 12500
no postcode for listing 12547
reverse geocodin

In [13]:
df_post = get_postcodes(df_post, geom_list_post, range(len(df_post)))

reverse geocoding listing 0
reverse geocoding listing 500
no postcode for listing 722
reverse geocoding listing 1000
reverse geocoding listing 1500
reverse geocoding listing 2000
reverse geocoding listing 2500
no postcode for listing 2759
reverse geocoding listing 3000
reverse geocoding listing 3500
reverse geocoding listing 4000
reverse geocoding listing 4500
no borough for listing 4872
reverse geocoding listing 5000
reverse geocoding listing 5500
no postcode for listing 5553
no postcode for listing 5970
reverse geocoding listing 6000
no postcode for listing 6069
reverse geocoding listing 6500
reverse geocoding listing 7000
no postcode for listing 7012
no borough for listing 7283
reverse geocoding listing 7500
reverse geocoding listing 8000
no borough for listing 8207
reverse geocoding listing 8500
reverse geocoding listing 9000
reverse geocoding listing 9500
reverse geocoding listing 10000
no postcode for listing 10047
reverse geocoding listing 10500
no postcode for listing 10699
rev

In [17]:
df_pre.to_csv(r'C:\Users\samih\OneDrive\Documents\STA 160\df_pre_clean.csv', index = False, header=True)
df_post.to_csv(r'C:\Users\samih\OneDrive\Documents\STA 160\df_post_clean.csv', index = False, header=True)

In [16]:
# # get counties and neighbourhoods for each listing
# # get locator
# locator = Nominatim(user_agent="myGeocoder")

# # get coordinates
# geom_pre = df_pre.latitude.map(str) + ", " + df_pre.longitude.map(str)
# geom_post = df_post.latitude.map(str) + ", " + df_post.longitude.map(str)

# # create columns for storage
# df_pre['county'] = 'NA'
# df_post['county'] = 'NA'
# df_pre['neigh'] = 'NA'
# df_post['neigh'] = 'NA'
# df_pre['borough'] = 'NA'
# df_post['borough'] = 'NA'
# df_pre['postcode'] = 'NA'
# df_post['postcode'] = 'NA'
# df_pre['rich'] = 'NA'
# df_post['rich'] = 'NA'

# # add counties and neighbourhoods
# # for pre dataset
# for i in range(len(geom_pre)):
#     address = locator.reverse(geom_pre[i]).raw['address']
    
#     if 'suburb' in address.keys():
#         df_pre.borough.iloc[i] = address['suburb']
#     else:
#         print("No suburb for pre listing {}".format(i))
    
#     if 'county' in address.keys():
#         df_pre.county.loc[i] = address['county']
#     elif 'city_district' in address.keys():
#         df_pre.county.loc[i] = address['city_district']
#     else:
#         print("No city district or county for pre listing {}".format(i))
        
#     if 'neighbourhood' in address.keys():
#         df_pre.neigh.loc[i] = address['neighbourhood']
#     else:
#         print("No neighbourhood for pre listing {}".format(i))
        
#     if 'postcode' in address.keys():
#         df_pre.postcode.loc[i] = address['postcode']
#         if address['postcode'] in rich_codes:
#             df_pre.rich.loc[i] = 1
#         else:
#             df_pre.rich.loc[i] = 0
#     else:
#         print("No postcode for post listing {}".format(i))
    
# # for post dataset
# for i in range(len(geom_post)):
#     address = locator.reverse(geom_post[i]).raw['address']
#     if 'suburb' in address.keys():
#         df_post.borough.iloc[i] = address['suburb']
#     else:
#         print("No suburb for post listing {}".format(i))
#     if 'county' in address.keys():
#         df_post.county.loc[i] = address['county']
#     elif 'city_district' in address.keys():
#         df_post.county.loc[i] = address['city_district']
#     else:
#         print("No city district or country for post listing {}".format(i))
        
#     if 'neighbourhood' in address.keys():
#         df_post.neigh.loc[i] = address['neighbourhood']
#     else:
#         print("No neighbourhood for post listing {}".format(i))
        
#     if 'postcode' in address.keys():
#         df_post.postcode.loc[i] = address['postcode']
#         if address['postcode'] in rich_codes:
#             df_post.rich.loc[i] = 1
#         else:
#             df_post.rich.loc[i] = 0
#     else:
#         print("No postcode for post listing {}".format(i))